超参调优(Hyperparameter Tuning)是机器学习中的一项重要任务，在选定一种模型后，需要找出最优的参数组合。一般机器学习的模型参数都有十几或者几十个，而深度学习则要考虑隐藏层，每层的节点数以及激活函数等，可调的参数更多。在这么多的参数中找到最优的组合，一方面要靠经验来确定方向和划定大致范围，另一方面就是拼算力，暴力算出最佳的参数组合。目前人工智能相关的工作一般有两类：数据工程师(Data Engineer)和数据科学家(Data Scientist)，前者重点是数据的收集和整理，后者则在建模和超参调优，有时也叫机器学习工程师(Machine Learning Engineer)。

云计算因其强大算力，天然适用于超参调优，AWS和Google也都推出了相关服务，并且把它们做为其云服务的重要卖点，如AWS Sagemaker，GCP ML Engine等。GCP一向高冷，其ML Engine据说目前超参调优的算法最好的，而AWS则更亲民，胜在整体和服务的丰富。本文主要介绍Sagemaker调优的原理和用法，后续还会使用ML Engine进行更深入的对比。

## AWS SageMaker
SageMaker是全流程机器学习服务，涵盖从数据分析/整理，建模及训练，超参调优到生产部署的全过程，用户只需要关注这些上层应用，而底层的基础设施则全由AWS接管。Sagemaker包含Notebook Instance，用户可以直接建Notebook服务器，在上面进行数据分析，尤其方便了团队间协作。在建模、训练和调参阶段，Sagemaker提供很多现成的算法供你直接调用，同时支持用户自定义的算法。训练出最优算法后，还可以直接部署并提供调用的Endpoint，与API Manager和Lambda结合后就可以生成可直接供生产环境使用的API.

SageMaker的参数调优服务使用Docker容器(Container)来安装算法的运行环境，AWS提供的算法都有现成的容器，如果想要使用自己的算法就需要按一定规则制作容器。超参调优时用户需要定义可调的超参数和取值范围，SageMaker使用Bayesian方法猜出一组参数，然后启动一个新的虚拟机(Instance)，装载算法容器后进行训练，训练结果要按一定格式告知Sagemaker，然后继续下一次训练，直到达到最大训练次数。Sagemaker会纪录训练结果并选出最优参数集，训练可以并行进行，由用户设置最大并行训练的任务数。

参数调优是按使用的虚拟机小时数收费的，尤其要注意每一次训练Sagemaker是使用一个新的虚拟机，而不是在相同的虚拟机中启动一个新的容器。当训练任务不及一个小时也是按一个小时收费，而且算法不能充分利用整个虚拟机的CPU和内存时也不能在这虚拟机器添加新的训练任务。当算法的规模比较大时，这个问题并不明显，但是如果是相对简单的算法而且数据集比较小时，就存在资源不能充分利用、价格过高的问题，这可能是Sagemaker需要改进的地方，在超参调优时可以让单个虚拟机支持多个训练任务。

## Sagemaker 自有算法参数调优

使用AWS提供的算法比较简单，但很多情况下，提供的算法往往不能满足需求，而使用自有算法则灵活很多，也有助于理解整个工作过程。AWS为了便于大家使用，提供了很多的样例，[scikit_bring_your_own](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)就提供了一个非常详细的使用案例。

第一步是制作Sagemaker可使用的容器镜像(Image)。Sagemaker在调用容器训练会传递一个参数"train"，如果容器中没有定义ENTRYPOINT，就会直接调用容器中的train函数，否则train就是ENTRYPOINT的第一个参数。传入的数据和参数会挂载在`/opt/ml/`目录下，按如下结构

    /opt/ml
    ├── input
    │   ├── config
    │   │   ├── hyperparameters.json
    │   │   └── resourceConfig.json
    │   └── data
    │       └── <channel_name>
    │           └── <input data>
    ├── model
    │   └── <model files>
    └── output
        └── failure

注意在单独的训练任务时`<channel_name>`是`training`，而在超参调优时则为`train`。制作完成容器镜像后，推到ECR中准备调用

超参按JSON格式放在hyperparameters.json中，读入后就可以按字典索引了。resourceConfig.json中有网络参数，用于分布式训练。训练完成的模型文件放在model文件夹中，如果训练失败则在output/failure写入失败的日志。

训练完成后要往stdout中输出一个衡量结果的指标，并且告诉sagemaker的格式，sagemaker在输出找到这个指标值并做为下一步训练的依据。比如输出

```
 print("Gradient Boosting score: {:.4f}\n".format(score.mean()))
```
就可以告诉检索格式为

```
objective_metric_name = 'score'
metric_definitions = [{'Name': 'score',
                       'Regex': 'score: ([0-9\\.]+)'}]
```

然后还需要定义超参的搜索范围，有三种方式——连续值、整数和类型值，这些完成就可以启动超参调优任务了。主要是下面几步：

1. 定义estimator，包括使用的容器镜像地址、虚拟机类型、输入数据和不需要调整的超参

```
estimator = sage.estimator.Estimator(image_name = image,
                       role = role, train_instance_count = 1, 
                       train_instance_type = 'ml.c4.xlarge',
                       input_mode  = 'File',
                       output_path="s3://{}/{}/output".format(bucket, prefix),
                       sagemaker_session=sess,
                       hyperparameters={'random_state': '2019'}     
                    )
```

2. 定义需要调优的超参范围

```
hyperparameter_ranges = {'max_depth': IntegerParameter(8, 32),
                         'learning_rate': ContinuousParameter(0.02, 1),
                         'n_estimators': IntegerParameter(2000, 4000),
                         'min_samples_split':  ContinuousParameter(0.001, 0.1),
                         'min_samples_leaf' : ContinuousParameter(0.001, 0.1),
                         'loss': CategoricalParameter(['huber', 'ls', 'quantile']), 
                         'max_features': CategoricalParameter(['sqrt', 'auto', 'log2']),
                                               
                        }
```
3. 定义超参调优任务，包括最大任务数和并行任务数

```
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            objective_type='Minimize',
                            max_jobs=500,
                            max_parallel_jobs=10)
```

4. 启动调优任务

```
data_location = "s3://{}/{}/all".format(bucket, prefix)
tuner.fit({'train': data_location})
```

具体代码请参照[scikit_bring_your_own](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)。
